# Copy an analysis job from a Flywheel Instance

Given the ID of a Flywheel job, this will create a python script to re-run it.  You can then edit and run that script.

The cell below will get a Flywheel client if you are logged in to a Flywheel instance.  It prints out the URL of the instance so you know where you are logged in.

In [95]:
import os
import stat
import pprint
from datetime import datetime

import flywheel
fw = flywheel.Client('')
fw.get_config().site.api_url

'https://ss.ce.flywheel.io/api'

In [194]:
def write_script_to_run_job(job_id):
    
    job = fw.get_job(job_id)
    gear = fw.get_gear(job.gear_id)
    print(f"gear.gear.name is {gear.gear.name}")
    destination_id = job.config["destination"]["id"]
    print(f"job's analysis destination_id is {destination_id}")
    analysis = fw.get_analysis(destination_id)
    print(f"job's analysis's parent's type is {analysis.parent['type']}")
    project_id = analysis.parents["project"]
    project = fw.get_project(project_id)
    project_label = project.label
    print(f"Project label: {project.label}")
    destination = fw.get(analysis.parent["id"])
    print(f"job's destination is {destination.label}")
    script_name = f"{project_label}_{analysis.parent['type']}_{destination.label}.py"
    print(f"Creating script: {script_name} ...\n")
    
    input_files = dict()
    for key, val in job.config["inputs"].items():
        file_name = val["location"]["name"]
        input_files[key] = file_name

    lines = f"""#! /usr/bin/env python3
'''Run {gear.gear.name} on {analysis.parent['type']} "{destination.label}"'''

import os
import argparse
from datetime import datetime


import flywheel


def main():

    fw = flywheel.Client('')
    print(fw.get_config().site.api_url)

    gear = fw.lookup("gears/{gear.gear.name}")
    print("gear.gear.version for job was = {gear.gear.version}")"""

    with open(script_name, "w") as sfp:
        for line in lines.split("\n"):
            sfp.write(line + "\n")
        sfp.write("    print(f\"gear.gear.version now = {gear.gear.version}\")\n")
        sfp.write(f"    print(\"destination_id = {analysis.parent['id']}\")\n")
        sfp.write(f"    print(\"destination type is: {analysis.parent['type']}\")\n")
        sfp.write(f"    destination = fw.get(\"{analysis.parent['id']}\")\n")
        sfp.write("\n")
        sfp.write(f"    project = fw.get_project(\"{project_id}\")\n")
        sfp.write("    inputs = dict()\n")
        sfp.write(f"    for key, val in {input_files}.items():\n")
        sfp.write("        inputs[key] = project.get_file(val)\n")
        sfp.write("\n")
        sfp.write(f"    config = {pprint.pformat(job['config']['config'], indent=4)}\n")
        sfp.write("\n")
        sfp.write("    now = datetime.now()\n")
        sfp.write("    analysis_label = (\n")
        sfp.write("        f'{gear.gear.name} {now.strftime(\"%m-%d-%Y %H:%M:%S\")} SDK launched'\n")
        sfp.write("    )\n")
        sfp.write("    print(f\"analysis_label = {analysis_label}\")\n")
        sfp.write("\n")
        sfp.write("    analysis_id = gear.run(\n")
        sfp.write("        analysis_label=analysis_label,\n")
        sfp.write("        config=config,\n")
        sfp.write("        inputs=inputs,\n")
        sfp.write("        destination=destination,\n")
        sfp.write("    )\n")
        sfp.write("    return analysis_id\n")
        sfp.write("\n")
        sfp.write("\n")
        sfp.write("if __name__ == '__main__':\n")
        sfp.write("\n")
        sfp.write("    parser = argparse.ArgumentParser(description=__doc__)\n")
        sfp.write("    args = parser.parse_args()\n")
        sfp.write("\n")
        sfp.write("    analysis_id = main()\n")
        sfp.write("\n")
        sfp.write("    print(f\"analysis_id = {analysis_id}\")\n")
        sfp.write("\n")
        sfp.write("    os.sys.exit(0)\n")

    os.system(f"black {script_name}")

    st = os.stat(script_name)
    os.chmod(script_name, st.st_mode | stat.S_IEXEC)

Get the job ID from the URL when you select the job of interest in the "Jobs Log" in the Flywheel UI.

In [195]:
write_script_to_run_job("601c74110533ee34c0708d3e")

gear.gear.name is bids-app-template
job's analysis destination_id is 601c74110533ee34c0708d3d
job's analysis's parent's type is session
Project label: BIDS_multi_session
job's destination is ses-Session2
Creating script: BIDS_multi_session_session_ses-Session2.py ...



In [196]:
write_script_to_run_job("601c24590533ee34c06fdda3")

gear.gear.name is bids-app-template
job's analysis destination_id is 601c24590533ee34c06fdda2
job's analysis's parent's type is session
Project label: BIDS_multi_session
job's destination is ses-Session1
Creating script: BIDS_multi_session_session_ses-Session1.py ...



In [154]:
gear = fw.lookup("gears/bids-app-template")
gear.gear.version

'0.0.0_0.14.0'

In [135]:
job = fw.get_job("601c24590533ee34c06fdda3")
for key, val in job.config["inputs"].items():
    file_name = val["location"]["name"]
    print(file_name)

license.txt


In [125]:
input_object = project.get_file('license.txt')
input_object

{'classification': {},
 'created': datetime.datetime(2019, 11, 22, 16, 39, 28, 610000, tzinfo=tzutc()),
 'deid_log_id': None,
 'hash': 'v0-sha384-4fb1e82ab91d0dc4e68aef2e091a1e854aaf67a6dd8e5a8ea8500f1ad65467dc625d46fea757acfcb3e9e3f1d0fb5dc6',
 'id': '21d79e68-561d-4fe2-9ae9-e4e6576f3d71',
 'info': {'BIDS': {'Filename': '',
                   'Folder': '',
                   'Path': '',
                   'error_message': "Filename '' is too short",
                   'ignore': False,
                   'template': 'project_file',
                   'valid': False}},
 'info_exists': True,
 'mimetype': 'text/plain',
 'modality': None,
 'modified': datetime.datetime(2019, 11, 22, 16, 39, 28, 170000, tzinfo=tzutc()),
 'name': 'license.txt',
 'origin': {'id': 'andyworth@flywheel.io',
            'method': None,
            'name': None,
            'type': 'user',
            'via': None},
 'replaced': None,
 'size': 58,
 'tags': [],
 'type': 'text',
 'zip_member_count': None}

In [126]:
input_object.ref

<bound method FileMixin.ref of {'classification': {},
 'created': datetime.datetime(2019, 11, 22, 16, 39, 28, 610000, tzinfo=tzutc()),
 'deid_log_id': None,
 'hash': 'v0-sha384-4fb1e82ab91d0dc4e68aef2e091a1e854aaf67a6dd8e5a8ea8500f1ad65467dc625d46fea757acfcb3e9e3f1d0fb5dc6',
 'id': '21d79e68-561d-4fe2-9ae9-e4e6576f3d71',
 'info': {'BIDS': {'Filename': '',
                   'Folder': '',
                   'Path': '',
                   'error_message': "Filename '' is too short",
                   'ignore': False,
                   'template': 'project_file',
                   'valid': False}},
 'info_exists': True,
 'mimetype': 'text/plain',
 'modality': None,
 'modified': datetime.datetime(2019, 11, 22, 16, 39, 28, 170000, tzinfo=tzutc()),
 'name': 'license.txt',
 'origin': {'id': 'andyworth@flywheel.io',
            'method': None,
            'name': None,
            'type': 'user',
            'via': None},
 'replaced': None,
 'size': 58,
 'tags': [],
 'type': 'text',
 'zip_m